In [1]:
from nltk.corpus import sentiwordnet as swn
from nltk import pos_tag
import json
import sys
import tqdm

In [2]:
# Read Data
with open('./sentenced-reviews.json') as data_file:
    data = data_file.readlines()

data = [x.strip() for x in data]
json_reviews = [json.loads(x) for x in data]

In [18]:
sentiment_scores = []
for review in tqdm.tqdm(json_reviews):
    review_score = 0   
    sentence_scores = []
    for sentence in review:
        sentence_score = 0
        
        for i, word_and_tag in enumerate(sentence):
            word = word_and_tag["word"]
            pos_tag = word_and_tag["posTag"]
            flag = 0
            new_tag = ""
            score = 0
            if (word == "unk"):
                continue
            
            try:
                if (i != (len(sentence) -1) and sentence[i+1]["word"] == "not"):
                    flag = 1
                elif (i != 0 and sentence[i-1]["word"] == "bad"):
                    flag = 1
                
                score = get_score(word, pos_tag, flag)
                
                if (score == None):
                    continue
                elif (score == -1):
                    new_tag = eng_tag(word)
                    score = get_score(word, new_tag, flag)
                
            except Exception as e:
#                 print(e)
                continue
            sentence_score += score
        sentence_scores.append(sentence_score)
    review_score = sum(x for x in sentence_scores)
    sentiment_scores.append(review_score)

100%|██████████| 10/10 [00:00<00:00, 2259.62it/s]

1
3
2
1
1
1
1
2
4
1


In [17]:
print(sentiment_scores)

[-0.75, -1.25, -0.125, -2.125, -0.875, -0.625, -1.25, 0.0, -0.125, -0.5]


In [11]:
sentence = json_reviews[0][0]
for i, word_and_tag in enumerate(sentence):
            word = word_and_tag["word"]
            pos_tag = word_and_tag["posTag"]
            flag = 0
            new_tag = ""
            score = 0
            if (word == "unk"):
                continue
            
            try:
                if (i != (len(sentence) -1) and sentence[i+1]["word"] == "not"):
                    flag = 1
                elif (i != 0 and sentence[i-1]["word"] == "bad"):
                    flag = 1
                
                score = get_score(word, pos_tag, flag)
                
                if (score == None):
                    continue
                elif (score == -1):
                    new_tag = eng_tag(word)
                    score = get_score(word, new_tag, flag)
                
                sentence_score += score
            except Exception as e:
#                 print(e)
                continue
print(sentence_score)

-1.25


In [7]:
first_sentence = json_reviews[0][0]
final_score = 0
for i, word_tag in enumerate(first_sentence):
    f_score = 0
    flags = 0
    f_word = word_tag["word"]
    f_tag = word_tag["posTag"]
    if (i != (len(first_sentence) -1) and first_sentence[i+1]["word"] == "not"):
        flags = 1
    
    f_score = get_score(f_word, f_tag, flags)
    
    if (f_score == -1):
        new_eng_tag = eng_tag(f_word)
        f_score = get_score(f_word, new_eng_tag, flags)
        
    final_score += f_score
print(final_score)

<class 'nltk.corpus.reader.sentiwordnet.SentiSynset'>
<class 'nltk.corpus.reader.sentiwordnet.SentiSynset'>
<class 'nltk.corpus.reader.sentiwordnet.SentiSynset'>
<class 'nltk.corpus.reader.sentiwordnet.SentiSynset'>
<class 'nltk.corpus.reader.sentiwordnet.SentiSynset'>
-1.375


In [4]:
def get_score(word, postag, next_not):
    
    try:
        if postag == "Noun":  # NOUN
            score = swn.senti_synset(word + '.n.01')
        elif postag == "Verb":  # VERB
            score = swn.senti_synset(word + '.v.01')
        elif postag == "Adjective":  # ADJECTIVE
            score = swn.senti_synset(word + '.a.01')
        elif postag == "Adverb":  # ADVERB
            score = swn.senti_synset(word + '.r.01')
    
    except Exception as e:
        return -1
    
#     if (score.obj_score() == 1.0):
# #         print (word)
#         return None
    print(type(score))
    true_score = score.pos_score() - score.neg_score()
    if (next_not == 1 and true_score > 0):
        true_score = true_score * (-1)
        
    return true_score

In [5]:
def calculate_one_score(scores):
    if len(scores) == 0:
        return 0.0
    
    numerator = 0.0
    denominator = 0.0
    for i in range(len(scores)):
        numerator += scores[i] * (1 / (i+2))
        denominator += 1 / (i+1)
    return numerator / denominator
    

In [6]:
def eng_tag(word):
    lst = [word]
    raw = pos_tag(lst)
    eng_pos = raw[0][1]
    if eng_pos == "NN":
        return "Noun"
    elif eng_pos == "VB":
        return "Verb"
    elif eng_pos == "RB":
        return "Adverb"
    elif eng_pos == "JJ":
        return "Adjective"
    else:
        return None